# 🛰️ Acceso a Datos NASA GES DISC con Python

## Descripción General

Este notebook demuestra cómo acceder a datos de NASA GES DISC (Goddard Earth Sciences Data and Information Services Center) usando Python y la librería oficial `earthaccess` recomendada por NASA.

### Características principales:
- ✅ **Autenticación moderna** usando `earthaccess`
- ✅ **Búsqueda inteligente** de datasets por coordenadas, fechas y tipos
- ✅ **Descarga directa** desde la nube de NASA
- ✅ **Visualización** de datos climáticos
- ✅ **Compatibilidad** con múltiples formatos (NetCDF, HDF)

### Datasets soportados:
- **IMERG**: Precipitación diaria y mensual
- **MERRA-2**: Datos atmosféricos
- **MODIS**: Temperatura superficial terrestre
- **GPM**: Datos de precipitación

### Referencias:
- [Tutoriales oficiales NASA GES DISC](https://github.com/nasa/gesdisc-tutorials)
- [Documentación earthaccess](https://earthaccess.readthedocs.io/)
- [Cómo acceder a datos GES DISC con Python](https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Access%20GES%20DISC%20Data%20Using%20Python)

## 1. 📦 Importar Librerías Requeridas

Primero instalamos e importamos todas las librerías necesarias para acceder a los datos de NASA.

In [ ]:
# Instalar earthaccess si no está disponible
import subprocess
import sys

try:
    import earthaccess
    print("✅ earthaccess ya está instalado")
except ImportError:
    print("📦 Instalando earthaccess...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "earthaccess"])
    import earthaccess

In [ ]:
# Importar todas las librerías necesarias
import earthaccess
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings

# Configurar visualización
plt.rcParams['figure.figsize'] = (12, 8)
warnings.filterwarnings('ignore')

print("✅ Librerías importadas exitosamente:")
print(f"   - earthaccess: {earthaccess.__version__}")
print(f"   - xarray: {xr.__version__}")
print(f"   - matplotlib: disponible")
print(f"   - cartopy: disponible")
print(f"   - pandas: {pd.__version__}")
print(f"   - numpy: {np.__version__}")

## 2. 🔐 Configurar Credenciales de Autenticación

La autenticación con NASA Earthdata es necesaria para acceder a los datos. `earthaccess` maneja esto automáticamente.

In [ ]:
# Configurar credenciales NASA Earthdata
# Opción 1: Usar credenciales ya guardadas en .netrc
try:
    auth = earthaccess.login(strategy="netrc")
    if auth.authenticated:
        print("✅ Autenticado usando archivo .netrc existente")
    else:
        raise Exception("No se pudo autenticar con .netrc")
except:
    # Opción 2: Autenticación interactiva (pedirá usuario y contraseña)
    print("🔐 Configurando autenticación interactiva...")
    auth = earthaccess.login(strategy="interactive", persist=True)
    
    if auth.authenticated:
        print("✅ Autenticación exitosa! Credenciales guardadas en .netrc")
    else:
        print("❌ Error en la autenticación. Verifica tus credenciales.")
        
# Mostrar información de autenticación
if auth.authenticated:
    print("🎯 Listo para acceder a datos NASA Earthdata")
else:
    print("⚠️  Necesitas credenciales válidas de https://urs.earthdata.nasa.gov/")

## 3. ⚙️ Configurar Parámetros de Acceso a Datos

Definimos los parámetros para buscar y acceder a diferentes tipos de datos climáticos.

In [ ]:
# Configuración de datasets y parámetros
datasets_config = {
    'IMERG_DAILY': {
        'short_name': 'GPM_3IMERGHH',  # Datos IMERG de precipitación
        'doi': '10.5067/GPM/IMERGDF/DAY/07',  # DOI para datos diarios
        'description': 'Precipitación diaria GPM IMERG',
        'variables': ['precipitationCal', 'lat', 'lon', 'time']
    },
    'MERRA2_SLV': {
        'short_name': 'M2T1NXSLV',  # MERRA-2 Single Level Variables
        'version': '5.12.4',
        'description': 'MERRA-2 Variables de Superficie',
        'variables': ['T2M', 'QV2M', 'PS', 'U10M', 'V10M']  # Temperatura, humedad, presión, viento
    }
}

# Coordenadas de interés (Península Ibérica)
locations = {
    'Madrid': {'lat': 40.4168, 'lon': -3.7038},
    'Barcelona': {'lat': 41.3851, 'lon': 2.1734},
    'Valencia': {'lat': 39.4699, 'lon': -0.3763},
    'Sevilla': {'lat': 37.3891, 'lon': -5.9845}
}

# Configurar área geográfica y temporal
bbox = (-10, 35, 5, 45)  # Península Ibérica (oeste, sur, este, norte)
fecha_inicio = '2024-03-01'  # Cambiar por fecha de interés
fecha_fin = '2024-03-03'     # Período de 3 días para prueba

print("🗺️  Configuración espacial:")
print(f"   Bounding Box: {bbox} (Península Ibérica)")
print(f"   Ciudades: {list(locations.keys())}")

print("📅 Configuración temporal:")
print(f"   Fecha inicio: {fecha_inicio}")
print(f"   Fecha fin: {fecha_fin}")

print("📊 Datasets configurados:")
for key, config in datasets_config.items():
    print(f"   - {key}: {config['description']}")

## 4. 🔍 Buscar y Descargar Archivos de Datos

Usamos `earthaccess` para buscar datos disponibles según nuestros criterios espaciales y temporales.

In [ ]:
# Función para buscar granulos de datos
def search_granules(dataset_name, temporal_range, bbox=None, max_results=5):
    """
    Buscar granulos usando earthaccess
    """
    config = datasets_config[dataset_name]
    
    search_params = {
        'temporal': temporal_range,
        'count': max_results
    }
    
    # Añadir parámetros específicos del dataset
    if 'short_name' in config:
        search_params['short_name'] = config['short_name']
    if 'version' in config:
        search_params['version'] = config['version']
    if 'doi' in config:
        search_params['doi'] = config['doi']
    if bbox:
        search_params['bounding_box'] = bbox
    
    print(f"🔍 Buscando {config['description']}...")
    print(f"   Parámetros: {search_params}")
    
    try:
        results = earthaccess.search_data(**search_params)
        print(f"✅ Encontrados {len(results)} granulos")
        return results
    except Exception as e:
        print(f"❌ Error en búsqueda: {e}")
        return []

# Buscar datos MERRA-2 (más probable que existan)
print("=" * 60)
granules_merra2 = search_granules('MERRA2_SLV', (fecha_inicio, fecha_fin), bbox)

# Mostrar información de los granulos encontrados
if granules_merra2:
    print(f"\\n📁 Información del primer granulo encontrado:")
    first_granule = granules_merra2[0]
    print(f"   ID: {first_granule['meta']['concept-id']}")
    print(f"   Nombre: {first_granule['umm']['GranuleUR']}")
    print(f"   Tamaño: {first_granule.size()} MB")
else:
    print("⚠️ No se encontraron granulos para el período especificado")

In [ ]:
# Opción 1: Streaming directo (recomendado para análisis rápido)
def stream_data(granules, variables=None):
    """
    Hace streaming de datos directamente a xarray sin descargar
    """
    print("🌊 Streaming datos a memoria...")
    try:
        # Abrir archivos usando earthaccess (streaming directo desde la nube)
        files = earthaccess.open(granules)
        
        # Cargar en xarray
        ds = xr.open_mfdataset(files, engine='h5netcdf', combine='by_coords')
        
        print(f"✅ Dataset cargado: {ds.sizes}")
        return ds
    except Exception as e:
        print(f"❌ Error en streaming: {e}")
        return None

# Opción 2: Descarga local (para archivos grandes o análisis offline)
def download_data(granules, local_path='./data'):
    """
    Descarga archivos localmente
    """
    print(f"💾 Descargando a {local_path}...")
    try:
        downloaded_files = earthaccess.download(granules, local_path=local_path)
        print(f"✅ Descargados {len(downloaded_files)} archivos")
        return downloaded_files
    except Exception as e:
        print(f"❌ Error en descarga: {e}")
        return []

# Probar streaming si hay datos disponibles
if granules_merra2:
    print("\\n" + "=" * 60)
    print("🚀 PROBANDO STREAMING DE DATOS...")
    
    # Usar solo el primer granulo para la prueba
    test_granules = granules_merra2[:1]
    
    dataset = stream_data(test_granules)
    
    if dataset is not None:
        print(f"\\n📊 Información del dataset:")
        print(f"   Dimensiones: {dict(dataset.sizes)}")
        print(f"   Variables disponibles: {list(dataset.data_vars.keys())}")
        print(f"   Coordenadas: {list(dataset.coords.keys())}")
else:
    print("\\n⏭️ Saltando streaming - no hay granulos disponibles")

## 5. 📋 Cargar e Inspeccionar Datos

Analizamos la estructura y metadatos de los datos cargados.

In [ ]:
# Función para inspeccionar el dataset
def inspect_dataset(ds, dataset_name="Dataset"):
    """
    Inspecciona un dataset de xarray y muestra información útil
    """
    print(f"\\n🔍 INSPECCIÓN DE {dataset_name.upper()}")
    print("=" * 50)
    
    # Información general
    print(f"📏 Dimensiones: {dict(ds.sizes)}")
    print(f"📊 Variables de datos: {len(ds.data_vars)}")
    print(f"🗺️  Coordenadas: {len(ds.coords)}")
    
    # Detalles de variables importantes
    print("\\n📋 Variables principales:")
    for var_name, var in ds.data_vars.items():
        if len(var.dims) >= 2:  # Solo variables multidimensionales
            attrs = dict(var.attrs) if var.attrs else {}
            units = attrs.get('units', 'N/A')
            long_name = attrs.get('long_name', 'N/A')
            print(f"   • {var_name}: {var.dims} | {units}")
            if long_name != 'N/A':
                print(f"     └─ {long_name}")
    
    # Información temporal
    if 'time' in ds.coords:
        time_coord = ds.coords['time']
        print(f"\\n⏰ Rango temporal:")
        print(f"   Inicio: {pd.to_datetime(time_coord.min().values)}")
        print(f"   Fin: {pd.to_datetime(time_coord.max().values)}")
        print(f"   Pasos: {len(time_coord)}")
    
    # Información espacial
    if 'lat' in ds.coords and 'lon' in ds.coords:
        lat_range = (float(ds.lat.min()), float(ds.lat.max()))
        lon_range = (float(ds.lon.min()), float(ds.lon.max()))
        print(f"\\n🌍 Cobertura espacial:")
        print(f"   Latitud: {lat_range[0]:.2f}° a {lat_range[1]:.2f}°")
        print(f"   Longitud: {lon_range[0]:.2f}° a {lon_range[1]:.2f}°")
        print(f"   Resolución: ~{abs(float(ds.lat[1] - ds.lat[0])):.3f}° lat, ~{abs(float(ds.lon[1] - ds.lon[0])):.3f}° lon")
    
    return ds

# Inspeccionar dataset si está disponible
if 'dataset' in locals() and dataset is not None:
    inspect_dataset(dataset, "MERRA-2")
    
    # Ejemplo de extracción de datos para una ubicación específica
    if 'lat' in dataset.coords and 'lon' in dataset.coords:
        print("\\n🎯 EXTRAYENDO DATOS PARA MADRID")
        madrid_coords = locations['Madrid']
        
        # Seleccionar punto más cercano a Madrid
        madrid_data = dataset.sel(
            lat=madrid_coords['lat'], 
            lon=madrid_coords['lon'], 
            method='nearest'
        )
        
        print(f"   Coordenadas seleccionadas: {float(madrid_data.lat.values):.2f}°, {float(madrid_data.lon.values):.2f}°")
        
        # Mostrar variables disponibles
        if 'T2M' in madrid_data.data_vars:
            temp_data = madrid_data['T2M']
            print(f"   Temperatura (T2M): {float(temp_data.mean()):.1f} K ({float(temp_data.mean()) - 273.15:.1f}°C)")
        
else:
    print("⏭️ No hay dataset cargado para inspeccionar")
    
    # Crear datos de ejemplo para demostración
    print("\\n🎬 Creando datos de ejemplo para demostración...")
    
    # Generar datos sintéticos
    times = pd.date_range('2024-03-01', '2024-03-03', freq='1D')
    lats = np.linspace(35, 45, 20)  # Península Ibérica
    lons = np.linspace(-10, 5, 30)
    
    # Crear datos de temperatura sintéticos
    temp_data = 15 + 10 * np.random.random((len(times), len(lats), len(lons)))
    
    # Crear dataset de ejemplo
    dataset = xr.Dataset({
        'temperature': (['time', 'lat', 'lon'], temp_data, {
            'long_name': 'Temperatura del aire a 2 metros (ejemplo)',
            'units': '°C'
        })
    }, coords={
        'time': times,
        'lat': lats,
        'lon': lons
    })
    
    print("✅ Dataset de ejemplo creado")
    inspect_dataset(dataset, "Ejemplo Sintético")

## 6. 📊 Procesar y Visualizar Datos

Creamos visualizaciones de los datos climáticos usando matplotlib y cartopy.

In [ ]:
# Función para crear mapas con proyección geográfica
def plot_spatial_data(ds, variable, time_index=0, title_prefix=""):
    """
    Crea un mapa de datos espaciales usando cartopy
    """
    try:
        # Seleccionar variable y tiempo
        if variable in ds.data_vars:
            data = ds[variable]
        else:
            # Usar la primera variable disponible
            data = ds[list(ds.data_vars.keys())[0]]
            variable = list(ds.data_vars.keys())[0]
        
        # Seleccionar tiempo si existe
        if 'time' in data.dims and len(data.time) > time_index:
            data = data.isel(time=time_index)
            time_str = pd.to_datetime(data.time.values).strftime('%Y-%m-%d')
            title = f"{title_prefix}{variable} - {time_str}"
        else:
            title = f"{title_prefix}{variable}"
        
        # Crear figura con proyección
        fig = plt.figure(figsize=(14, 10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        
        # Añadir características geográficas
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS)
        ax.add_feature(cfeature.LAND, alpha=0.3)
        ax.add_feature(cfeature.OCEAN, alpha=0.3)
        
        # Plot principal
        im = data.plot(
            ax=ax, 
            transform=ccrs.PlateCarree(),
            cmap='viridis',
            add_colorbar=False
        )
        
        # Añadir colorbar
        cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
        units = data.attrs.get('units', '')
        cbar.set_label(f"{variable} {units}")
        
        # Añadir ciudades
        for city, coords in locations.items():
            ax.plot(coords['lon'], coords['lat'], 'ro', markersize=8, transform=ccrs.PlateCarree())
            ax.text(coords['lon'], coords['lat'], f'  {city}', 
                   transform=ccrs.PlateCarree(), fontsize=10, fontweight='bold')
        
        # Configurar extent para España
        ax.set_extent([-12, 6, 34, 46], ccrs.PlateCarree())
        ax.gridlines(draw_labels=True, alpha=0.5)
        
        plt.title(title, fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Error en visualización: {e}")
        # Fallback: plot simple sin proyección
        if variable in ds.data_vars:
            data = ds[variable]
            if 'time' in data.dims:
                data = data.isel(time=0)
            data.plot(figsize=(12, 6))
            plt.title(f"{title_prefix}{variable}")
            plt.show()

# Función para series temporales
def plot_time_series(ds, location_name, variables=None):
    """
    Crea gráficos de series temporales para una ubicación
    """
    if location_name not in locations:
        print(f"❌ Ubicación '{location_name}' no encontrada")
        return
    
    coords = locations[location_name]
    
    try:
        # Seleccionar punto más cercano
        if 'lat' in ds.coords and 'lon' in ds.coords:
            point_data = ds.sel(lat=coords['lat'], lon=coords['lon'], method='nearest')
        else:
            print("⚠️ No se encontraron coordenadas lat/lon")
            return
        
        # Seleccionar variables
        if variables is None:
            variables = list(point_data.data_vars.keys())[:3]  # Primeras 3 variables
        
        # Crear subplots
        fig, axes = plt.subplots(len(variables), 1, figsize=(12, 4*len(variables)))
        if len(variables) == 1:
            axes = [axes]
        
        fig.suptitle(f'Series Temporales - {location_name}', fontsize=16, fontweight='bold')
        
        for i, var in enumerate(variables):
            if var in point_data.data_vars:
                data = point_data[var]
                
                if 'time' in data.dims:
                    data.plot(ax=axes[i])
                    axes[i].set_title(f"{var} ({data.attrs.get('units', '')})")
                    axes[i].grid(True, alpha=0.3)
                else:
                    axes[i].text(0.5, 0.5, f'Variable {var}\\nSin dimensión temporal', 
                               ha='center', va='center', transform=axes[i].transAxes)
                    axes[i].set_title(f"{var} (valor: {float(data.values):.2f})")
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Error en serie temporal: {e}")

# Visualizar datos si están disponibles
if 'dataset' in locals() and dataset is not None:
    print("🎨 CREANDO VISUALIZACIONES...")
    
    # Mapa espacial
    try:
        plot_spatial_data(dataset, list(dataset.data_vars.keys())[0], title_prefix="NASA GES DISC - ")
    except Exception as e:
        print(f"⚠️ Error en mapa espacial: {e}")
    
    # Serie temporal para Madrid
    try:
        plot_time_series(dataset, 'Madrid', list(dataset.data_vars.keys())[:2])
    except Exception as e:
        print(f"⚠️ Error en serie temporal: {e}")
    
    print("✅ Visualizaciones completadas")
else:
    print("📊 No hay dataset disponible para visualizar")
    print("💡 Ejecuta las celdas anteriores para cargar datos")

## 🎯 Ejemplo Práctico: Análisis Completo

Ahora combinamos todo lo anterior en un flujo de trabajo completo para analizar datos climáticos.

In [ ]:
# Análisis completo: Buscar, cargar y visualizar datos
def complete_analysis(dataset_type='MERRA2_SLV', region='spain', days=3):
    """
    Flujo de trabajo completo para análisis de datos NASA
    """
    print("🚀 INICIANDO ANÁLISIS COMPLETO DE DATOS NASA GES DISC")
    print("=" * 60)
    
    # 1. Configurar parámetros
    end_date = datetime.now() - timedelta(days=30)  # Datos de hace 1 mes (más probable que existan)
    start_date = end_date - timedelta(days=days)
    
    temporal_range = (start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
    
    print(f"📅 Periodo de análisis: {temporal_range[0]} a {temporal_range[1]}")
    print(f"🗺️  Región: {region}")
    print(f"📊 Dataset: {datasets_config[dataset_type]['description']}")
    
    # 2. Buscar datos
    print("\\n🔍 PASO 1: Búsqueda de granulos...")
    granules = search_granules(dataset_type, temporal_range, bbox, max_results=3)
    
    if not granules:
        print("❌ No se encontraron datos para el período especificado")
        print("💡 Sugerencias:")
        print("   - Prueba con fechas más antiguas")
        print("   - Verifica tu autenticación")
        print("   - Cambia el tipo de dataset")
        return None
    
    # 3. Cargar datos
    print("\\n🌊 PASO 2: Carga de datos...")
    ds = stream_data(granules[:2])  # Usar solo 2 granulos para eficiencia
    
    if ds is None:
        print("❌ Error al cargar datos")
        return None
    
    # 4. Procesar y analizar
    print("\\n📋 PASO 3: Análisis de datos...")
    inspect_dataset(ds, datasets_config[dataset_type]['description'])
    
    # 5. Crear visualizaciones
    print("\\n🎨 PASO 4: Visualizaciones...")
    
    # Mapa espacial
    variable_list = list(ds.data_vars.keys())
    if variable_list:
        plot_spatial_data(ds, variable_list[0], title_prefix="NASA - ")
    
    # Series temporales para ciudades principales
    for city in ['Madrid', 'Barcelona']:
        try:
            plot_time_series(ds, city, variable_list[:1])
        except Exception as e:
            print(f"⚠️ No se pudo crear serie temporal para {city}: {e}")
    
    print("\\n✅ ANÁLISIS COMPLETO FINALIZADO")
    return ds

# Ejecutar análisis completo
try:
    result_dataset = complete_analysis()
    
    if result_dataset is not None:
        print("\\n🎊 ¡Análisis exitoso!")
        print("💾 El dataset está disponible como 'result_dataset'")
        print("🔄 Puedes modificar fechas y volver a ejecutar")
    else:
        print("\\n⚠️ Análisis no completado")
        print("🔧 Revisa configuración y autenticación")
        
except Exception as e:
    print(f"❌ Error en análisis: {e}")
    print("🛠️ Verifica tu conexión a internet y credenciales")

## 🔗 Recursos Adicionales y Próximos Pasos

### 📚 Documentación y Tutoriales
- [Tutoriales oficiales NASA GES DISC](https://github.com/nasa/gesdisc-tutorials)
- [Documentación earthaccess](https://earthaccess.readthedocs.io/)
- [Guía de acceso a datos GES DISC](https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Access%20GES%20DISC%20Data%20Using%20Python)

### 🛠️ Funcionalidades Avanzadas
- **OPeNDAP**: Acceso directo a subconjuntos de datos sin descarga completa
- **Cloud Computing**: Procesamiento en AWS junto a los datos
- **Bulk Downloads**: Descarga masiva de series temporales largas
- **API Subsetting**: Extracción específica por regiones y variables

### 💡 Casos de Uso Comunes
- **Análisis climatológico**: Series temporales de precipitación y temperatura
- **Validación de modelos**: Comparación con datos observacionales
- **Estudios de impacto**: Análisis de eventos extremos
- **Monitoreo ambiental**: Seguimiento de cambios a largo plazo

### ⚡ Optimización de Rendimiento
- Usa `bounding_box` para limitar la región de interés
- Limita el rango temporal para reducir volumen de datos
- Considera usar `streaming` en lugar de descarga para análisis exploratorio
- Aprovecha el paralelismo de `dask` para datasets grandes